In [ ]:
import feedparser
from newspaper import Article
import html

"""
Every minute;
1. Fetch feed for all RSS URLs. Filter out feed data for last 30 min.
2. Maintain a cache with TTL of 60 min. If the current feed title does not exist in the cache, push it and add it to processinglist
3. Filter the processinglist to find unique titles
4. For each of this title, analyse the news

"""


def get_summary(url):
    article = Article(url)
    article.download()
    article.parse()
    #print("Authors:", article.authors)
    return article.text

def get_articles(url):
    feed = feedparser.parse(url)
    for entry in feed.entries:
        title = entry.title
        #description = entry.summary
        published_date = entry.published
        #text = get_summary(entry.links[0]['href'])
        print(f"{published_date} - {html.unescape(title)}")
        #print(f"Published: {published_date}")
        #print(f"Text: {text}")
        #print("----------------------------------------------------------------------------------------------------")

get_articles('https://www.businesstoday.in/rssfeeds/?id=home')
get_articles('https://rss.app/feeds/tgll9PXJFaUoCjGV.xml')